## Ejercicio ML y NLP

Usando el dataset de **`movie_data.csv`**:

1. Aplica preprocesamiento de **NLP** a la columna de **`review`**.
    - Define una función que elimine signos de puntuación y que transforme todo a minúsculas, esta función recibe un string y debe retornar un string.
    - Define una función que haga tokenización y elimine stopwords, esta función recibe un string y debe retornar una lista de strings (tokens).
    - Define una función que haga Porter Stemmer, esta función recibe una lista de strings (tokens) y retorna un solo string.
2. Aplica los modelos de **Bag-of-Word** y **TF-IDF**.
3. Entrena modelos de clasificación para predecir la columna **`sentiment`**.
    - Define una función que tome como parámetros una lista de modelos de clasificación y que retorne un DataFrame con las métricas para cada modelo y el tiempo de ejecución de cada modelo.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Normalizacion
from sklearn.preprocessing import MinMaxScaler

# GridSearchCV
from sklearn.model_selection import GridSearchCV

# Train, Test
from sklearn.model_selection import train_test_split

# Metricas
from sklearn.metrics import jaccard_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

# Clasificadores
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import RadiusNeighborsClassifier
from sklearn.neighbors import NearestCentroid
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier

# Validacion
from sklearn.model_selection import StratifiedKFold

# Signos de Puntuacion

import string 
from nltk.tokenize import word_tokenize
import nltk
from nltk.stem import PorterStemmer

from sklearn.feature_extraction.text import CountVectorizer

from sklearn.feature_extraction.text import TfidfTransformer


In [2]:
df = pd.read_csv("movie_data.csv")


In [3]:
def quitar_morralla(serie):
    
    serie = serie.apply(lambda x: x.lower())
    stopwords = nltk.corpus.stopwords.words("English")
    stopwords.extend(['br', "''",'...',"n't","'s","'ll",'``',"''", "'ve","....",".....","--","'d", "'m"])
    filtered_fila = []
    for element in serie:
        element = word_tokenize(element)
        filtered_word = []
        for i in element:
            if len(i) > 4:
                if i not in stopwords:
                    filtered_word.append(i)
        filtered_fila.append(filtered_word)
   
    return filtered_fila

In [4]:

token_limpios= quitar_morralla(df["review"])
token_limpios

[['teenager',
  'martha',
  'moxley',
  'maggie',
  'grace',
  'moves',
  'high-class',
  'belle',
  'greenwich',
  'connecticut',
  'mischief',
  'night',
  'halloween',
  'murdered',
  'backyard',
  'house',
  'murder',
  'remained',
  'unsolved',
  'twenty-two',
  'years',
  'later',
  'writer',
  'fuhrman',
  'christopher',
  'meloni',
  'former',
  'detective',
  'fallen',
  'disgrace',
  'perjury',
  'simpson',
  'trial',
  'moved',
  'idaho',
  'decides',
  'investigate',
  'partner',
  'stephen',
  'weeks',
  'andrew',
  'mitchell',
  'purpose',
  'writing',
  'locals',
  'squirm',
  'welcome',
  'support',
  'retired',
  'detective',
  'steve',
  'carroll',
  'robert',
  'forster',
  'charge',
  'investigation',
  'discover',
  'criminal',
  'power',
  'money',
  'cover',
  'murder.',
  'murder',
  'greenwich',
  'movie',
  'story',
  'murder',
  'fifteen',
  'years',
  'committed',
  'wealthy',
  'teenager',
  'whose',
  'mother',
  'kennedy',
  'powerful',
  'family',
  'inf

In [5]:
stemmer = PorterStemmer()

def Porter_Stemmer_t(token_limpios):
    cadena_limpia = []
    for lista_token in token_limpios:
            cadena_limpia.append(" ".join([stemmer.stem(word) for word in lista_token]))
    return cadena_limpia

resultado = Porter_Stemmer_t(token_limpios)

In [6]:
"""set_resultado = set()
for r in resultado:
    set_resultado = set_resultado.union(set(r.split()))
    
len(set_resultado)"""

'set_resultado = set()\nfor r in resultado:\n    set_resultado = set_resultado.union(set(r.split()))\n    \nlen(set_resultado)'

In [7]:
"""set_resultado"""

'set_resultado'

In [8]:
# Inicializamos un objeto CountVecrtorizer()
count_vectorizer = CountVectorizer()


resultado = count_vectorizer.fit_transform(resultado)

resultado

# bag es una matriz sparse, por lo que Python decide mostrarla de la siguiente manera

<50000x75653 sparse matrix of type '<class 'numpy.int64'>'
	with 3493062 stored elements in Compressed Sparse Row format>

In [9]:
# TF-IDF


# Inicializamos un objeto Tfidf
tfidf = TfidfTransformer()

# Cambio la precisión de python a 2 decimales
np.set_printoptions(precision = 2)



# Entrenamos el Tfidf y transformamos la variable bag
resultado = tfidf.fit_transform(resultado)

resultado

<50000x75653 sparse matrix of type '<class 'numpy.float64'>'
	with 3493062 stored elements in Compressed Sparse Row format>

In [10]:
X_train, X_test, y_train, y_test = train_test_split(resultado, df["sentiment"].values, test_size = 0.3, random_state = 42)

print(f"X_train: {X_train.shape}, y_train: {y_train.shape}")
print(f"X_test: {X_test.shape},  y_test: {y_test.shape}")

X_train: (35000, 75653), y_train: (35000,)
X_test: (15000, 75653),  y_test: (15000,)


In [ ]:
%%time

# GaussianNB()

model = GaussianNB()

# Entrenamiento
model.fit(X_train.toarray(), y_train)

# Predicciones
yhat = model.predict(X_test.toarray())

# Métricas
print("Jaccard Index:", jaccard_score(y_test, yhat, average = "macro"))
print("Accuracy:"     , accuracy_score(y_test, yhat))
print("Precisión:"    , precision_score(y_test, yhat, average = "macro"))
print("Sensibilidad:" , recall_score(y_test, yhat, average = "macro"))
print("F1-score:"     , f1_score(y_test, yhat, average = "macro"))
print("ROC AUC:"      , roc_auc_score(y_test, yhat))